In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\vivek\\Documents\\GSC\\chatbot\\law_chatbot_gen-ai\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vivek\\Documents\\GSC\\chatbot\\law_chatbot_gen-ai'

In [5]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
%pwd

'c:\\Users\\vivek\\Documents\\GSC\\chatbot\\law_chatbot_gen-ai'

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 14248


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:

#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\vivek\AppData\Local\Temp\ipykernel_19540\67698624.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\vivek\anaconda3\envs\chatbot_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [ ]:
import pinecone
import os

# Initialize Pinecone the correct way
pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Define the index name
index_name = "lawbot"

# List existing indexes (optional)
print(pc.list_indexes())


{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'lawbot-t6cl0l6.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'lawbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [21]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is the punishment for rape?")
retrieved_docs

[Document(id='473a28ff-fb01-43d5-8934-0172c0241027', metadata={'page': 1376.0, 'page_label': '1377', 'source': 'Data\\ipc.pdf'}, page_content='commits rape shall be punished with imprisonment of either description for a term which shall\nnot be less than seven years but which may be for life or for a term which may extend to ten\nyears and shall also be liable to ﬁne unless the women raped is his own wife and is not under\ntwelve years of age, in which cases, he shall be punished with imprisonment of either\ndescription for a term which may extend to two years or with ﬁne or with both:'),
 Document(id='8f3eec53-4e49-46ed-b8ee-83231fc4cc2e', metadata={'page': 152.0, 'page_label': '153', 'source': 'Data\\ipc.pdf'}, page_content="501, 502).\n(11) Uttering any word, or making any sound or gesture, with an intention to insult the\nmodesty of a woman (section 509).\n(12) Misconduct in a public place by a drunken person (section 510).61.\n[s 53.9] Imprisonment for the remainder of the Accused

In [ ]:
from langchain_groq import ChatGroq
## Load the Groq API key
groq_api_key = os.getenv("GROQ_API_KEY", "key")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [64]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# Define your system prompt
system_prompt = (
    "You are a Legal Assistant designed to provide guidance and answer questions related to legal matters. "
    "Use the following pieces of retrieved context to address the user's query directly and comprehensively. "
    "If the context provides sufficient information, offer a clear and actionable response to the user's question. "
    "Ensure the response is professional, accurate, and easy to understand. "
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [65]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [70]:
response = rag_chain.invoke({"input": "punishments for traffic "})
print(response["answer"])

According to section 279 of the Indian Penal Code (IPC), rash driving or riding on a public way is punishable with imprisonment of either description for a term which may extend to six months, or with fine which may extend to one thousand rupees, or with both.


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'google.generativeai.generative_models.GenerativeModel'>

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
].
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
].
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_I

InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]